# Analyzing Wanderers

Important: This notebook uses [Plotly](https://plotly.com/python/) mostly to visualize data. Plotly charts won't be rendered in Githubs notebook viewer. So please [view this notebook on nbviewer](https://nbviewer.org/github/ymyke/abalyzer/blob/main/fidenzas.ipynb) (or on your local installation after cloning the repo). Note that nbviewer caches notebooks and sometimes takes a long time to pick up a new version.

On the upside, all the Plotly charts are interactive. You can zoom, hover, pan, etc. Zooming is especially important due to the outliers in some the data so you can zoom in on the interesting parts.

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Get the assets

In [ ]:
import datetime
from opensea import retrieve_assets

ids = list(range(0, 8888))
assets = retrieve_assets(token_ids=ids, contract="0x8184a482A5038B124d933B779E0Ea6e0fb72F54E")
print(datetime.datetime.now())


## Build a dataframe

Check the trait types there are. We'll see that 1) different assets can have different numbers of traits, 2) the same asset can have traits with the same name, i.e. "Celestial Body".

In [ ]:
[", ".join(t["trait_type"] for t in a["traits"]) for a in assets][:10]

All the possible celestial bodies there are:

In [ ]:
set([t["value"]  for a in assets for t in a["traits"] if t["trait_type"] == "Celestial Body"])

We will count the number of Celestial Bodies as well as the number of stars and galaxies when tabulating all the information into a dataframe:

(Large vs not large galaxies will be ignored; could be added as some point in the future.)

In [ ]:
from typing import Union
from functools import reduce
import operator
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)


def turn_assets_into_df_generalized(assets: list) -> Union[pd.DataFrame, list]:

    trait_blueprint = {
        k: "-" for k in set([t["trait_type"] for a in assets for t in a["traits"]])
    }
    trait_blueprint["Stars"] = 0
    trait_blueprint["Galaxies"] = 0
    trait_blueprint["Celestial Bodies"] = 0
    del trait_blueprint["Celestial Body"]

    _tt = []
    for a in assets:

        # Calc probability score:
        prob_score = (
            reduce(operator.mul, [t["trait_count"] / 1000 for t in a["traits"]], 1)
            * 1000
        )

        # Get current price:
        price = price_symbol = np.nan
        if a["sell_orders"] is not None:
            so = a["sell_orders"][0]
            price = float(so["current_price"]) / 10 ** int(
                so["payment_token_contract"]["decimals"]
            )
            price_symbol = so["payment_token_contract"]["symbol"]
            # Convert to ETH if necessary:
            if price_symbol not in ["ETH", "WETH"]:
                price *= float(so["payment_token_contract"]["eth_price"])
                price_symbol = "ETH"

        # Get last price:
        last_price = last_price_symbol = last_sale_date = np.nan
        if a["last_sale"] is not None:
            ls = a["last_sale"]
            last_price = float(ls["total_price"]) / 10 ** int(
                ls["payment_token"]["decimals"]
            )
            last_price_symbol = ls["payment_token"]["symbol"]
            # Convert to ETH if necessary:
            if last_price_symbol not in ["ETH", "WETH"]:
                last_price *= float(ls["payment_token"]["eth_price"])
                last_price_symbol = "ETH"
            last_sale_date = ls["event_timestamp"]

        # Traits:
        traits = trait_blueprint.copy()
        for t in a["traits"]:
            if t["trait_type"] == "Celestial Body":
                traits["Celestial Bodies"] += 1
                if "Galaxy" in t["value"]:
                    traits["Galaxies"] += 1
                else:
                    traits["Stars"] += 1
            else:
                if traits[t["trait_type"]] == "-":
                    traits[t["trait_type"]] = t["value"]
                else:
                    traits[t["trait_type"]] += ", " + t["value"]

        # Add everything to the table:
        _tt.append(
            [
                a["token_id"],  # CHANGED
                a[
                    "token_id"
                ],  # FIXME Not really a nice solution // but they don't have names and I need some attribute that is not the index to expose via hover in the Plotly charts.
                price,
                price_symbol,
                last_price,
                last_price_symbol,
                last_sale_date,
                prob_score,
                *[  # All the traits:
                    v for _, v in sorted(traits.items(), key=lambda x: x[0])
                ],
                a["permalink"],
            ]
        )

    # Get the traits names:
    trait_names = [  # All the traits:
        k for k, _ in sorted(trait_blueprint.items(), key=lambda x: x[0])
    ]

    # Turn table into df:
    df = pd.DataFrame(_tt)
    df.columns = [
        "ID",
        "Name",
        "Price",
        "Psymbol",
        "Lastprice",
        "LPsymbol",
        "LPdate",
        "Probscore",
        *trait_names,
        "Link",
    ]
    df["LPdate"] = pd.to_datetime(df.LPdate)
    df = df.set_index("ID").sort_values("Lastprice", ascending=False)

    return df, traits


In [ ]:
import numpy as np
import pandas as pd
df, traits = turn_assets_into_df_generalized(assets)
hoverdata = [ "Name", *traits, "Probscore"]

In [ ]:
assert set(df.LPsymbol.unique()) == set(["ETH", "WETH", np.nan]), """
This sheet does not do currency conversion at the moment and therefore 
assumes all prices are in (W)ETH. But there are more symbols in the input 
data which would lead to apples being compared to ranges below. Aborting.
"""

## How many pieces have a last price on OpenSea?

In [ ]:
print(f"{df[~df.Lastprice.isna()].shape[0] / df.shape[0]:.0%}")

## How are prices evolving over time?

Note that this only takes into account the last sale of each piece.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 8)
pd.plotting.register_matplotlib_converters()

fig = px.scatter(df, x="LPdate", y="Lastprice", hover_data=hoverdata)
fig.show()

## Price levels today

In [ ]:
fig = px.histogram(df, x="Lastprice", marginal="box", hover_data=hoverdata)
fig.show()

## How frequent are the different traits?

In [ ]:
pd.set_option("display.max_columns", None)

from IPython.display import display
for trait in sorted(traits.keys()):
    display(
        pd.DataFrame(df[trait].value_counts(normalize=False, sort=True, ascending=True)).transpose()
    )


## How much do people pay for the different traits?

In [ ]:
for trait in sorted(traits.keys()):

    fig = go.Figure()
    for traitvariant in (
        df.groupby(trait).median().sort_values("Lastprice", ascending=False).index
    ):
        fig.add_trace(
            go.Box(
                y=df[df[trait] == traitvariant].fillna("-").Lastprice.values,
                name=traitvariant,
                boxpoints="all",
                jitter=0.2,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1,
            )
        )
    fig.update_layout(title=trait)
    fig.show()

**Some Observations**:

- The monolith anomaly commands much higher prices than other anomalies. It is also the most seldom anomaly.
- The more celestial bodies, the higher the price. Same with galaxies and stars. (4 stars is really seldom and outperforms 0-3 stars massively.)
- A cat cockpit slightly outperforms other cockpits. Interestingly, glyphs perform average, even though they are much rarer than average.
- Arms: Joints perform best, followed by ETH, bong, porn, alien joint, and skull. Flag is clearly worst (I wonder why?). Bongs perform much better than their rarity would suggest, whereas flags left perform much worse than their rarity would suggest (not so much for flags right, strangely).
- Music doesn't make a difference.
- Panels: Crypto, bounty, tetris, spam are the best preformers which is very much in line with their rarity.
- Planet: Anger orb and destroyed planet are clearly performing best and also rarest.
- Ships: Doesn't make much of a difference.
- Window: Doesn't make much of a difference.
- Special: Specials in general add to the value. Warp squad sixteen performs very high.

## What is pricier, galaxies or stars?

There seems to be no fundamental difference between stars and galaxies. In either case, rarity is the differentiator: 3 galaxies is slightly rare, 4 stars is rare, 5 bodies overall is rare and 6 bodies is superrare – so these all command higher prices.

In [ ]:
df["Celestialstype"] = df.apply(
    lambda x: "Starsonly"
    if x["Galaxies"] == 0
    else "Galaxiesonly"
    if x["Stars"] == 0
    else "Both",
    axis=1,
)
px.box(df, x="Celestial Bodies", y="Lastprice", color="Celestialstype", points="all", hover_data=hoverdata)